Olá! Vamos iniciar mais um projeto de análise de dados!

Dessa vez temos conosco um dataset interessantíssimo fornecido pela Olist, nele contamos com mais de 100 mil informações sobre os pedidos feitos entre 2016 e 2018 em diversas lojas virtuais do Brasil, as informações vão desde geolocalização, status dos pedidos, até às avaliações feitas pelos clientes.

Segundo a própria Olist todas as informações que constam em nosso dataset são reais, sendo mantido apenas em anonimato a identidade dos clientes, como forma de preservação. Ainda segundo a empresa outras referências foram substituídas pelos nomes das casas referentes à série Game of Thrones. 

Antes de começarmos as nossas análises propriamente, convém nos atentarmos ao fornecedor de nossos dados:

* "Em termos gerais, Olist é uma plataforma que conecta a sua loja virtual ao seu público na internet. Funciona como um facilitador de processos, uma conexão segura entre lojistas e consumidores. Olist é um grande descomplicador para qualquer lojista – online ou offline – que deseja vender nos principais marketplaces do Brasil."


In [3]:
import pandas_profiling 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


np.random.seed(789)
%matplotlib inline
pd.set_option('display.max_columns', 100)
plt.style.use('fivethirtyeight')

In [4]:
geo= pd.read_csv('olist_geolocation_dataset.csv')

* Este estudo é voltado para a Geolocalização.
* Vejamos de onde são os nossos principais clientes!

In [5]:
import folium
from folium import plugins

In [6]:
#Apenas os dados longitudinais e latitudinais.

coordenadas = geo.loc[:,['geolocation_lat', 'geolocation_lng']]

In [7]:
mp1 = folium.Map(location=[-21.764089543486026, -48.167850377601376], tiles='CartoDB dark_matter', zoom_start=4.3)
mp1.add_child(plugins.HeatMap(coordenadas, radius=10))


mp1